<a href="https://colab.research.google.com/github/merucode/Stock/blob/11_stock_info/21_theme_report(dust).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0. Install Dependency

In [ ]:
!pip install pykrx

# STEP 1. Infomation about Issue

### Google 검색어 트랜드
* [Google 검색어 트랜드](https://trends.google.co.kr/trends/explore?date=2011-01-01%202023-10-24&geo=KR&q=%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80&hl=ko)

### Issue(일자 뉴스 서울미세먼지수치)
* 2017-03-24 [미세먼지 ‘매우 나쁨’… 붉은 조명의 남산타워
](https://www.seoul.co.kr/news/newsView.php?id=20170118500151)(130)
* 2018-03-25 [전국 미세먼지 나쁨 예보, 노약자·임산부·기저질환자 더 주의](https://m.health.chosun.com/svc/news_view.html?contid=2018032301083)(140)
* 2019-03-05 [오늘 서울 관측사상 최악 초미세먼지 가능성…150㎍/㎥ 육박](https://www.yna.co.kr/view/AKR20190305081600004)(200)
* 2021-03-29 [서울 미세먼지 경보·초미세먼지 주의보 동시 발령](https://www.chosun.com/national/transport-environment/2021/03/29/KUG6O5YU6BHIFF3HVYE6ZI4VR4/)(300)
* 2023-01-08 [미세먼지 감옥에 대한민국 갇혔다](https://www.seoul.co.kr/news/newsView.php?id=20230109001012)(150)
* 2023-03-23 [오후부터 황사 유입…서울 등 서쪽 중심 미세먼지 ‘매우나쁨’](https://www.seoul.co.kr/news/newsView.php?id=20230323500018)(150)

### Reference Site
* [서울특별시 대기환경정보 미세먼지통계-일별](https://cleanair.seoul.go.kr/statistics/dayAverage)

# Plot

In [ ]:
from pykrx import stock           # 데이터 : 주식 데이터 전송
import datetime                   # 그래프 : 문자열 시간 전환
import matplotlib.pyplot as plt   # 그래프
%matplotlib inline

tickers_KOSDAQ = stock.get_market_ticker_list(market="KOSDAQ")
tickers_KOSPI = stock.get_market_ticker_list(market="KOSPI")
tickers = tickers_KOSDAQ + tickers_KOSPI
ticker_dict = dict()

for ticker in tickers:
    stock_name = stock.get_market_ticker_name(ticker)
    ticker_dict[stock_name] = ticker

def convert_ohlcv_format(df):
    df = df.reset_index()
    df = df[['날짜', '시가', '고가', '저가', '종가', '거래량']]
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    return df

def plot_with_issue_date(start_date, end_date, stock_name_lst, issue_date_lst,  year_lst, ticker_dict,adjust=False):
    ### Get ohlcv
    ohlcv_lst = []
    for stock_name in stock_name_lst:
        ohlcv = stock.get_market_ohlcv(start_date, end_date, ticker_dict[stock_name])
        ohlcv = convert_ohlcv_format(ohlcv)
        ohlcv_lst.append(ohlcv)
    ohlcv_kosdaq = stock.get_index_ohlcv(start_date, end_date, "2001")
    ohlcv_kosdaq = convert_ohlcv_format(ohlcv_kosdaq)
    ohlcv_kosdaq['close'] = ohlcv_kosdaq['close'] if adjust==True else 1

    ### Convert str date to datetime date
    issue_date_dt_lst = []    # issue_date
    for issue_date_str in issue_date_lst:
        issue_date_dt_lst.append(datetime.datetime.strptime(issue_date_str, '%Y-%m-%d'))

    year_dt_lst = []    # year
    for year_str in year_lst:
        year_dt_lst.append(datetime.datetime.strptime(year_str, '%Y-%m-%d'))


    ### Plot
    fig, ax = plt.subplots(len(stock_name_lst)+1, 1, figsize=(12, 12), constrained_layout=True)
    # figsize : Plot 창 크기
    # constrained_layout=True : 서브플롯 간의 간격 자동 조절

    plot_color_lst = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']

    for ohlcv, stock_name in zip(ohlcv_lst, stock_name_lst):     # close price line plot
        ax[0].plot(ohlcv['date'], ohlcv['close']/ohlcv['close'][0]/ohlcv_kosdaq['close'], label=stock_name)
        ax[0].set_title('종합 그래프')

    for i, ohlcv in enumerate(ohlcv_lst):     # close price adjust by kosdaq line plot
        ax[i+1].plot(ohlcv['date'], ohlcv['close']/ohlcv_kosdaq['close'], color=plot_color_lst[i])
        ax[i+1].set_title(f'{stock_name_lst[i]}')

    for pt in issue_date_dt_lst:    # add issue date line
        for i in range(0, len(stock_name_lst)+1):
            ax[i].axvline(x=pt, color='#D62728')

    for pt in year_dt_lst:    # add year line
        for i in range(0, len(stock_name_lst)+1):
            ax[i].axvline(x=pt, color='#EFE034')

    ax[0].legend()
    plt.show()

In [ ]:
start_date = '2017-01-01'
end_date = '2023-10-18'
stock_name_lst = ['크린앤사이언스', '하츠']
year_lst = ['2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01']
issue_date_lst = ['2017-03-24', '2018-03-25', '2019-03-05', '2021-03-29', '2023-01-08','2023-03-23']

plot_with_issue_date(start_date, end_date, stock_name_lst, issue_date_lst, year_lst, ticker_dict)

#
: